# train problem


### baseline 문제

 - run 과정에서 Datafarame object가 그대로 넘어가서 not callable 문제 발생.
 - train, model 부분에서 모든 문제가 발생하는 부분을 찾았는데 도저히 찾지 못함.
 - 학습 진행 과정 안에 print(1 ~)를 넣어서 숫자별로 어느 위치까지 학습에서 문제가 없는지 찾았는데 어느 구간이 아니라 일정 횟수에서 문제가 발생.
 - 뭔가 코드 자체의 문제가 아닌 것으로 보여 런타임 재시작 하고 파이토치 캐시를 모두 삭제하고 진행하니 문제없이 진행.

---

 - vram 문제로 여러번 문제가 일어났는데 모델을 확인하기 위해서 예제로 푼 경우에 모델이 여러개 로드 되어 문제 발생.
 - 모두 정리해도 vram문제가 해결되지 않아서 batch size를 8까지 줄임.
 - 그래도 문제가 생겨 colab에서 standard 모드에서 premium 모드로 변경.

### softmax 문제

 - 처음에는 소프트맥스 함수를 nn.Sequential() 안에 넣어서 softmax 함수가 정의되지 않았다는 에러가 발생.
 - forward 과정에서 소프트맥스 함수를 넣어서 문제 해결

---

Expected floating point type for target with class probabilities, got Long

---

 - 위의 에러는 loss함수 과정에서 발생.

---

tensor([[0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 0, 1],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0],
        [0, 1, 0, 0]], device='cuda:0')
tensor([[0.0609, 0.0633, 0.0604, 0.0610],
        [0.0633, 0.0635, 0.0619, 0.0669],
        [0.0616, 0.0656, 0.0631, 0.0641],
        [0.0632, 0.0614, 0.0602, 0.0620],
        [0.0649, 0.0602, 0.0663, 0.0632],
        [0.0644, 0.0631, 0.0614, 0.0645],
        [0.0624, 0.0612, 0.0644, 0.0630],
        [0.0659, 0.0581, 0.0616, 0.0642],
        [0.0568, 0.0633, 0.0635, 0.0610],
        [0.0637, 0.0618, 0.0651, 0.0598],
        [0.0621, 0.0647, 0.0613, 0.0603],
        [0.0603, 0.0613, 0.0608, 0.0636],
        [0.0632, 0.0590, 0.0644, 0.0594],
        [0.0641, 0.0666, 0.0617, 0.0641],
        [0.0604, 0.0641, 0.0619, 0.0615],
        [0.0628, 0.0628, 0.0619, 0.0613]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

---

 - 위와 같이 batch size 16으로 잡아서 정답 레이블과 예측 레이블이 나오는데 첫번째 레이블인 type에 대한 정보다.
 - (16, 4) size로 동일한 형태로 나왔는데 어떤 점에서 문제가 일어나 에러가 발생했는지는 pytorch docs를 통해서 파악 필요.
 - loss 함수 계산에서 서로 데이터 type이 같지 않아서 문제 발생.
 - 정답 레이블에 .float()를 사용하여 문제를 해결.

 => 성능이 완전 바닥으로 나왔는데 아마 학습이 옳바로 되지 않거나 레이블 디코딩 과정에서 문제가 있는 것으로 추측.
    아마도 인코딩은 torch one_hot을 사용하고 디코딩에서는 단순 파이썬으로 풀어버려서 둘 사이의 오차가 있어 디코딩에서 문제 발생으로 추정.
    일단 최적 hyperparameter와 모델을 찾는 작업을 먼저 진행하고 softmax를 사용한 분류기는 나중에 성능한계에 도달했을 때 다시 시도.

### pooler 문제

 - https://dacon.io/competitions/official/236027/codeshare/7277?page=1&dtype=recent
 - 위 링크의 코드를 많이 참고하여 진행하였는데 비슷한 모델을 사용하였고 마지막 pooler에서 identity layer로 변경한 것으로 고성능에 도달한 것이 인상적이었다.
 - 이는 랜덤성을 위해서 치환하였다고 하였는데 해당 레이어로 치환하는 것이 랜덤성을 올리는 이유와 이것이 성능향상에 어떻게 도움을 주는지 이해가 가지 않았다.
 - 일단 모델을 돌려 보면서 의문점을 같이 조사하였다.
 - 하지만 pooler를 치환하면 output shape가 변경되어 loss 함수에서 막혔는데 분명 pooler만 바뀌고 classifier 부분은 그대로인데 모양이 변한다는 점이 특이했다.
 - 참고한 코드에서는 확인하니 모델의 pooler를 nn.Identity()로 변경하고 이후에 모델의 학습 과정에서 동일한 레이어를 한번 더 사용하였다.
 - 동일한 레이어 구성을 하고 학습을 실행한 결과 성공적으로 학습이 이루어졌고 기존 점수와 동일한 성능을 보여 성능 향상을 이루지는 못했다.

### layer freeze

 - fine tuning 과정에서 예전에 layer weight의 freezing 퍼센티지에 따라서 학습 효율이 더 좋거나 오히려 freeze가 없는 경우 더 좋다는 논문을 본 기억이 있었다.
 - 또한 이전에 국비 학원에서 진행한 팀 프로젝트에서 청경채 생장 예측 대회를 진행하는데 layer freezing의 정도를 조절하는 것에 따라서 성능 차이가 났었다.
 - 당시에는 물론 이미지 데이터셋이어서 nlp와는 다른 데이터 형태지만 사전학습된 bert 계열 모델을 완성된 feature extractor로 가정하여 진행해 보았다.
 - 성능 향상이 이루어지지 않았다.
 - 이전까지 pytorch에서 layer의 weight를 조작하는 방법을 몰랐는데 이번 기회에 알게 되었다.
 - https://github.com/huggingface/transformers/issues/400
 -

### tqdm, zip 함수 문제

 - 각 레이블 column 기준으로 데이터셋을 4개로 나누어 4개의 동일한 모델에 넣어서 분류의 성능 향상을 노림.
 - 이 과정에서 train class에서 model에 4개의 data loader를 사용하기 위해서 zip 함수로 묶었는데 작동하지 않음.
 - 이유는 tqdm, enumerate는 매개변수가 len()을 가지지 않을 경우 작동하지 않기 때문이다.
 - https://beausty23.tistory.com/207?category=367726
 - 위의 레퍼런스를 통해서 해당 문제 해결.
 - zip 함수를 제일 밖으로 빼고 tqdm을 안으로 넣어서 해결.